In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from google.colab import drive
drive.mount('/content/gdrive')
folder_path = '/content/gdrive/My Drive/Colab Notebooks/Data 144 Kaggle Challenge/'
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, Bidirectional
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.layers import Dropout
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk import word_tokenize
STOPWORDS = set(stopwords.words('english'))

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


Using TensorFlow backend.


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [0]:
train = pd.read_csv(folder_path+'yelp_train.csv')

In [0]:
df = train
df = df.reset_index(drop=True)
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    text = BAD_SYMBOLS_RE.sub('', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing. 
    text = text.replace('x', '')
#    text = re.sub(r'\W+', '', text)
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
    return text

In [0]:
df['cleaned'] = df['text'].apply(clean_text)

In [5]:
# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 50000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 250
# This is fixed.
EMBEDDING_DIM = 100

tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(df['cleaned'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 287264 unique tokens.


In [6]:
X = tokenizer.texts_to_sequences(df['cleaned'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)

Shape of data tensor: (240000, 250)


In [7]:
Y = pd.get_dummies(df['is_good_rating']).values
print('Shape of label tensor:', Y.shape)

Shape of label tensor: (240000, 2)


In [8]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(192000, 250) (192000, 2)
(48000, 250) (48000, 2)


In [9]:
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH))
model.add(SpatialDropout1D(0.2))
model.add(Bidirectional(LSTM(200, dropout=0.2, recurrent_dropout=0.2)))
model.add(Dense(2, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 250, 100)          5000000   
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 250, 100)          0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 400)               481600    
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 802       
Total params: 5,482,402
Trainable params: 5,482,402
Non-trainable params: 0
_________________________________________________________________
None


In [0]:
batch_size = 128
model.fit(X, Y, epochs=1, batch_size=batch_size,validation_split=0.5)
drive.mount('/content/gdrive')
folder_path = '/content/gdrive/My Drive/Colab Notebooks/Data 144 Kaggle Challenge/'
test = pd.read_csv(folder_path+'yelp_test.csv')
test = test.reset_index(drop=True)
test['cleaned'] = test['text'].apply(clean_text)
X_test = tokenizer.texts_to_sequences(test['cleaned'].values)
X_test = pad_sequences(X_test, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X_test.shape)
preds = model.predict(X_test)
print(preds.shape)
predictions = preds.argmax(axis=1)
print(predictions[:5])
sample = pd.read_csv(folder_path+'yelp_sample.csv')
sample.is_good_rating = predictions
sample.set_index('review_id', inplace=True)
sample.to_csv('preds_bilstm1.csv')
!cp preds_bilstm1.csv /content/gdrive/My\ Drive/Colab\ Notebooks/Data\ 144\ Kaggle\ Challenge

Train on 120000 samples, validate on 120000 samples
Epoch 1/1
  9856/120000 [=>............................] - ETA: 1:07:04 - loss: 0.2186 - acc: 0.9136

In [0]:
model.fit(X, Y, epochs=1, batch_size=batch_size*2,validation_split=0.5)
drive.mount('/content/gdrive')
folder_path = '/content/gdrive/My Drive/Colab Notebooks/Data 144 Kaggle Challenge/'
test = pd.read_csv(folder_path+'yelp_test.csv')
test = test.reset_index(drop=True)
test['cleaned'] = test['text'].apply(clean_text)
X_test = tokenizer.texts_to_sequences(test['cleaned'].values)
X_test = pad_sequences(X_test, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X_test.shape)
preds = model.predict(X_test)
print(preds.shape)
predictions = preds.argmax(axis=1)
sample = pd.read_csv(folder_path+'yelp_sample.csv')
sample.is_good_rating = predictions
sample.set_index('review_id', inplace=True)
sample.to_csv('preds_bilstm2.csv')
!cp preds_bilstm2.csv /content/gdrive/My\ Drive/Colab\ Notebooks/Data\ 144\ Kaggle\ Challenge